<a href="https://colab.research.google.com/github/nikhilkarnwal/projects/blob/master/deeplearning_101(MNSIT%20using%20Pytorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
!gunzip *.gz
!ls

--2019-11-30 20:26:16--  http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘train-images-idx3-ubyte.gz’

train-images-idx3-u 100%[===================>]   9.45M  6.84MB/s    in 1.4s    

2019-11-30 20:26:17 (6.84 MB/s) - ‘train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2019-11-30 20:26:19--  http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 216.165.22.6
Connecting to yann.lecun.com (yann.lecun.com)|216.165.22.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28881 (28K) [application/x-gzip]
Saving to: ‘train-labels-idx1-ubyte.gz’

train-labels-idx1-u 100%[===================>]  28.20K  --.-KB/s    in 0.09s   

2019-11-30 20:26:19 (330 KB/s) - ‘train-labels-id

In [10]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
x = torch.randn(5, 3).to(device)
y = torch.randn(5, 3, device=device)
z = x + y
print(z)

cuda
tensor([[ 0.4553, -0.5451, -2.4367],
        [-0.8844,  0.8735,  0.0250],
        [ 0.9752,  0.4694, -2.8714],
        [ 0.0366, -0.9532, -0.9102],
        [ 1.5993, -0.8419, -2.8299]], device='cuda:0')


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np
import torch

def load(dataset="training", path=None):
    
    import os
    import struct

    if path is None:
        path = '/datasets/MNIST'
        if not os.path.isdir(path):
            path = './MNIST'
    if not os.path.isdir(path):
        raise ValueError("Cannot find dataset at '%s'" % path)

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows * cols)

    img = np.moveaxis(img, 0, -1)
    lbl = lbl.astype(int)

    return img, lbl


def show(image):
    """
    Render a given MNIST image provided as a column vector.

    Arguments:
        image (array): an array of shape (28*28) or (28, 28) representing a
            grey level image of size 28 x 28. Values are expected to be in the
            range [0, 1].

    Example:
        x, lbl = load(dataset="training", path="/datasets/MNIST")
        show(x[:, 0])
    """
    from matplotlib import pyplot
    import matplotlib as mpl

    rows = 28
    cols = 28
    if image.shape[0] != rows * cols and image.shape[0] * image.shape[1] != rows * cols:
        raise "the input is not an MNIST image."
    fig = pyplot.figure()
    ax = fig.add_subplot(1, 1, 1)
    image = image.reshape(rows, cols)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    pyplot.show()

def normalize(x):
    return np.tanh(x)

def label2onehot(lbl):
    d = np.zeros((lbl.max() + 1, lbl.size))
    d[lbl, np.arange(lbl.size)] = 1
    return d

def load_and_process(set='training'):
    x, y = load(set,".")
    print(x.shape)
    print(y.shape)
    x_norm = normalize(x.astype(np.float32))
    y_onehot = label2onehot(y)
    x_final = np.moveaxis(x_norm.reshape((28,28,1,-1)),[0,1,2,3],[-2,-1,-3,-4])
    y_final = np.moveaxis(y_onehot,[0,1],[-1,-2])
    print(x_final.shape, y_final.shape)
    return (x,y),(x_final, y_final)

In [5]:
(x,ytrain),(xtrain,_) = load_and_process("training")
(_,ytest),(xtest,_) = load_and_process('testing')

(784, 60000)
(60000,)
(60000, 1, 28, 28) (60000, 10)
(784, 10000)
(10000,)
(10000, 1, 28, 28) (10000, 10)


In [0]:
xtrain = torch.from_numpy(xtrain)
xtest = torch.from_numpy(xtest)
ltrain = torch.from_numpy(ytrain).to(dtype=torch.long)
ltest= torch.from_numpy(ytest).to(dtype=torch.long)

In [7]:
#26
import torch.nn as nn
import torch.nn.functional as F
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256,120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x ):
        size = x.size()[1:]
        return np.prod(size)
    
net = LeNet()
print(net)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [0]:
#29, 30
def backprop_deep(xtrain, ltrain, net, T, B=100, gamma=.001, rho=.9):
    N = xtrain.size()[0] # Training set size
    NB = B # Number of minibatches
    batch_size = int(N/NB);
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=rho)
    for epoch in range(T):
        running_loss = 0.0
        shuffled_indices = torch.randint(0,N,(N,))
        for k in range(NB):
            # Extract k-th minibatch from xtrain and ltrain
            minibatch_indices = shuffled_indices[k*batch_size:(k+1)*batch_size]
            inputs = xtrain[minibatch_indices,:,:,:]
            labels = ltrain[minibatch_indices]
            # Initialize the gradients to zero
            optimizer.zero_grad()
            # Forward propagation
            outputs = net(inputs)
            # Error evaluation
            loss = criterion(outputs, labels)
            # Back propagation
            loss.backward()
            # Parameter update
            optimizer.step()
            # Print averaged loss per minibatch every 100 mini-batches
            # Compute and print statistics
            with torch.no_grad():
                running_loss += loss.item()
            if k % 100 == 99:
                print('[%d, %5d] loss: %.3f' %
                (epoch + 1, k + 1, running_loss / 100))
                running_loss = 0.0
                
#net = LeNet()
#backprop_deep(xtrain, ltrain, net, T=3, B=600)

In [0]:
#31
with torch.no_grad():
    yinit = net(xtest)
    _, lpred = yinit.max(1)
    print(100 * (ltest == lpred).float().mean())

tensor(91.9500)


In [11]:
#32
xtrain = xtrain.to(device)
ltrain = ltrain.to(device)
xtest = xtest.to(device)
ltest = ltest.to(device)
net = LeNet().to(device)
backprop_deep(xtrain, ltrain, net, T=10, B=600)
model_path = './model_net.pth'
torch.save(net.state_dict(), model_path)

[1,   100] loss: 2.303
[1,   200] loss: 2.301
[1,   300] loss: 2.298
[1,   400] loss: 2.294
[1,   500] loss: 2.290
[1,   600] loss: 2.283
[2,   100] loss: 2.273
[2,   200] loss: 2.251
[2,   300] loss: 2.201
[2,   400] loss: 2.034
[2,   500] loss: 1.495
[2,   600] loss: 0.884
[3,   100] loss: 0.651
[3,   200] loss: 0.543
[3,   300] loss: 0.463
[3,   400] loss: 0.442
[3,   500] loss: 0.393
[3,   600] loss: 0.367
[4,   100] loss: 0.347
[4,   200] loss: 0.303
[4,   300] loss: 0.292
[4,   400] loss: 0.270
[4,   500] loss: 0.269
[4,   600] loss: 0.242
[5,   100] loss: 0.246
[5,   200] loss: 0.213
[5,   300] loss: 0.226
[5,   400] loss: 0.212
[5,   500] loss: 0.215
[5,   600] loss: 0.197
[6,   100] loss: 0.191
[6,   200] loss: 0.182
[6,   300] loss: 0.184
[6,   400] loss: 0.159
[6,   500] loss: 0.160
[6,   600] loss: 0.155
[7,   100] loss: 0.153
[7,   200] loss: 0.155
[7,   300] loss: 0.155
[7,   400] loss: 0.146
[7,   500] loss: 0.145
[7,   600] loss: 0.142
[8,   100] loss: 0.150
[8,   200] 

In [15]:
#33
n_net = LeNet().to(device)
n_net.load_state_dict(torch.load(model_path,map_location=device))
n_net.eval()
with torch.no_grad():
    yinit = n_net(xtest)
    _, lpred = yinit.max(1)
    print(100 * (ltest == lpred).float().mean())

tensor(96.9600, device='cuda:0')
